# Hello! and welcome to my Typhoon fine tuning notebook.


Hello! and welcome to my Typhoon fine tuning notebook.

Learn more about the model: https://arxiv.org/abs/2312.13951

# Install requirements

In [1]:

import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Installs Unsloth, Xformers (Flash Attention) and all other packages!

!pip install datasets peft accelerate bitsandbytes

# Clean up the notebook
clean_notebook()

Notebook cleaned.


# Set up environment variables

This is set with kaggle secret collection. If you're runing with other enviroment, they can be set .env

In [2]:
import os
os.environ['HF_TOKEN']         ="code"


# Select dataset

In [3]:
dataset_id = "Thaweewat/thai-med-pack"

# Load tokenizer and model

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


model_id = "scb10x/llama-3-typhoon-v1.5-8b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

# Set up trainable parameters

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["lm_head","all-linear"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4235264 || all params: 4544835584 || trainable%: 0.09318849761936734


In [7]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [8]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (act_fn): SiLU()
     

# Preprocess dataset

I parsed into `# Instruction: # Input: # Response:` also I added `<answer></answer>` xml and response but you may modify it. I just like this way :D

In [9]:
import os
import re
from datasets import load_dataset

data = load_dataset(dataset_id)


def parse(text: str) -> str:
    try:
        question_search = re.search(r'\[INST\](.*)\[/INST\]', text, re.IGNORECASE)
        question = question_search.group(1).strip()


        answer_search = re.search(r'\[/INST\](.*)\</s\>', text, re.IGNORECASE)
        answer = answer_search.group(1).strip()
        spec = f"""<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Act as a doctor and response the input question from a patient in Thai language with XML format <answer>Your answer</answer>
### Input:
{question}
### Response:
<answer> {answer} </answer>
</s>"""
    except:
        print(text)
        raise
    return {
        "spec": spec
    }


# parse(data["train"]["text"][0])
# data.map(lambda )
data = data.map(lambda samples: parse(samples["text"]), batch_size=8,num_proc=os.cpu_count())
data = data.map(lambda samples: tokenizer(samples["spec"]), batched=True, batch_size=8,num_proc=os.cpu_count())

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'spec', 'input_ids', 'attention_mask'],
        num_rows: 189190
    })
})

In [11]:
from IPython.display import Markdown, display


idx = 5

display(Markdown(f"**text:**\n\n{data['train'][idx]['text']}"))
display(Markdown("---"))
display(Markdown(f"**spec:**\n\n{data['train'][idx]['spec']}"))


**text:**

<s>[INST] สวัสดีค่ะ ช่วงเวลาประมาณ 3 สัปดาห์ หนูรู้สึกท้อแท้ สิ้นหวังกับทุกสิ่งทุกอย่าง ถูกกดดันอยู่ตลอดเวลา ทั้งที่บ้านและโรงเรียน หนูไม่เคยบอกความรู้สึกกับใคร เหมือนอยู่ตัวคนเดียว รู้สึกกลัวกับทุกอย่าง เวลาอยู่คนเดียวแล้วมันรู้สึกเหงา คิดเรื่องต่างๆนาๆ ร้องให้ออกมาทุกครั้ง มีวันหนึ่งหนูทนไม่ไหวเลยเล่าให้เพื่อนคนหนึ่งฟังและร้องให้ออกมาหนักมาก แต่มันก็ไม่ดีขึ้นเลย รู้สึกเหนื่อยตลอดเวลา หนูลองทำแบบทดสอบแล้วได้ 14 อาการแบบนี้เข้าข่ายโรคซึมเศร้าไหมค่ะ#รบกวนด้วยน่ะค่ะ [/INST] สวัสดีค่ะ จากที่เล่ามา คิดว่า มีภาวะทางอารมณ์ไม่ปกติ อาจมีภาวะเครียด หรืออาจมีภาวะโรคซึมเศร้าค่ะผู้ป่วยโรคซึมเศร้าจะมีอาการที่เปลี่ยนแปลงทั้งอารมณ์ ความรู้สึกนึกคิด ความจำ ความสัมพันธ์กับคนรอบข้างเปลี่ยนไป และอาการแสดงทางร่างกายต่างๆ โดยอาการอาจเปลี่ยนแปลงอย่างรวดเร็วเป็นสัปดาห์ หรือค่อยเป็นค่อยไปเป็นก็ได้-อาการทางอารมณ์ เช่น เศร้า หดหู่ อ่อนไหว บางคนอาจมีความรู้สึกไม่แจ่มใส เบื่อหน่าย บางคนมีอาการหงุดหงิด ฉุนเฉียวได้-ความรู้สึกนึกคิด เช่น มองอะไรแย่หมด ท้อแท้หมดหวัง รู้สึกไร้ค่า ไร้ความสามารถ บางคนอาจมีอารมณ์ชั่ววูบอยากทำร้ายตนเอง-สมาธิความจำแย่ลง-ความสัมพันธ์กับคนรอบข้างเปลี่ยนไป อาจจะเก็บตัว ไม่ค่อยพูด ทำงานแย่ลง-อาการแสดงทางร่างกาย เช่น อ่อนเพลีย ไม่มีแรง นอบหลับๆตื่นๆ ไม่เจริญอาหาร น้ำหนักลด ท้องผูก ท้องอืด ปวดหัว ปวดตามตัว*คนไข้หากมีอาการมากขึ้น อาจมีอาการหลงผิด ประสาทหลอน หูแว่วได้*โรคซึมเศร้านี้ยังมีลักษณะอาการคล้ายโรคอื่นอีก เช่น โรควิตกกังวล โรคอารมณ์แปรปรวน โรคสมองอักเสบ โรคไทรอยด์ทำงานบกพร่อง เป็นต้น-การรักษา: ในรายที่เป็นมาก แพทย์จะให้ยาช่วย แต่บางรายที่เป็นไม่มากแพทย์อาจจะพูดคุยให้คำแนะนำเพื่อให้อาการดีขึ้นดังนั้นจึงแนะนำให้ ควรเล่าถึงภาวะทางอารมณ์ให้คนใกล้ชิดหรือคนในครอบครัวฟังเพื่อให้อารมณ์ผ่อนคลายและช่วยแก้ไขปัญหานะคะและแนะนำว่าควรไปพบแพทย์เพื่อรับการวินิจฉัยและได้รับการรักษาอย่างเหมาะสมขอเป็นกำลังใจให้นะคะ </s>

---

**spec:**

<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Act as a doctor and response the input question from a patient in Thai language with XML format <answer>Your answer</answer>
### Input:
สวัสดีค่ะ ช่วงเวลาประมาณ 3 สัปดาห์ หนูรู้สึกท้อแท้ สิ้นหวังกับทุกสิ่งทุกอย่าง ถูกกดดันอยู่ตลอดเวลา ทั้งที่บ้านและโรงเรียน หนูไม่เคยบอกความรู้สึกกับใคร เหมือนอยู่ตัวคนเดียว รู้สึกกลัวกับทุกอย่าง เวลาอยู่คนเดียวแล้วมันรู้สึกเหงา คิดเรื่องต่างๆนาๆ ร้องให้ออกมาทุกครั้ง มีวันหนึ่งหนูทนไม่ไหวเลยเล่าให้เพื่อนคนหนึ่งฟังและร้องให้ออกมาหนักมาก แต่มันก็ไม่ดีขึ้นเลย รู้สึกเหนื่อยตลอดเวลา หนูลองทำแบบทดสอบแล้วได้ 14 อาการแบบนี้เข้าข่ายโรคซึมเศร้าไหมค่ะ#รบกวนด้วยน่ะค่ะ
### Response:
<answer> สวัสดีค่ะ จากที่เล่ามา คิดว่า มีภาวะทางอารมณ์ไม่ปกติ อาจมีภาวะเครียด หรืออาจมีภาวะโรคซึมเศร้าค่ะผู้ป่วยโรคซึมเศร้าจะมีอาการที่เปลี่ยนแปลงทั้งอารมณ์ ความรู้สึกนึกคิด ความจำ ความสัมพันธ์กับคนรอบข้างเปลี่ยนไป และอาการแสดงทางร่างกายต่างๆ โดยอาการอาจเปลี่ยนแปลงอย่างรวดเร็วเป็นสัปดาห์ หรือค่อยเป็นค่อยไปเป็นก็ได้-อาการทางอารมณ์ เช่น เศร้า หดหู่ อ่อนไหว บางคนอาจมีความรู้สึกไม่แจ่มใส เบื่อหน่าย บางคนมีอาการหงุดหงิด ฉุนเฉียวได้-ความรู้สึกนึกคิด เช่น มองอะไรแย่หมด ท้อแท้หมดหวัง รู้สึกไร้ค่า ไร้ความสามารถ บางคนอาจมีอารมณ์ชั่ววูบอยากทำร้ายตนเอง-สมาธิความจำแย่ลง-ความสัมพันธ์กับคนรอบข้างเปลี่ยนไป อาจจะเก็บตัว ไม่ค่อยพูด ทำงานแย่ลง-อาการแสดงทางร่างกาย เช่น อ่อนเพลีย ไม่มีแรง นอบหลับๆตื่นๆ ไม่เจริญอาหาร น้ำหนักลด ท้องผูก ท้องอืด ปวดหัว ปวดตามตัว*คนไข้หากมีอาการมากขึ้น อาจมีอาการหลงผิด ประสาทหลอน หูแว่วได้*โรคซึมเศร้านี้ยังมีลักษณะอาการคล้ายโรคอื่นอีก เช่น โรควิตกกังวล โรคอารมณ์แปรปรวน โรคสมองอักเสบ โรคไทรอยด์ทำงานบกพร่อง เป็นต้น-การรักษา: ในรายที่เป็นมาก แพทย์จะให้ยาช่วย แต่บางรายที่เป็นไม่มากแพทย์อาจจะพูดคุยให้คำแนะนำเพื่อให้อาการดีขึ้นดังนั้นจึงแนะนำให้ ควรเล่าถึงภาวะทางอารมณ์ให้คนใกล้ชิดหรือคนในครอบครัวฟังเพื่อให้อารมณ์ผ่อนคลายและช่วยแก้ไขปัญหานะคะและแนะนำว่าควรไปพบแพทย์เพื่อรับการวินิจฉัยและได้รับการรักษาอย่างเหมาะสมขอเป็นกำลังใจให้นะคะ </answer>
</s>

# Training

## Train

In [12]:
import transformers
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Clear the GPU cache
torch.cuda.empty_cache()

# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Define the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Set up the training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=9,  # Adjusted batch size
    gradient_accumulation_steps=5,  # Adjust gradient accumulation steps
    max_steps=35,
    learning_rate=1e-5,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit",
    load_best_model_at_end=True,
    save_strategy="no",
   
    
    
)

# Define the trainer
trainer = Trainer(
    model=model,
    train_dataset=data["train"],
    args=training_args,
    data_collator=data_collator

)

# # Disable caching to silence warnings (enable for inference)
model.config.use_cache = False

# # Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

# Train the model
trainer.train()


/home/tuchsanai/anaconda3/envs/llm/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


KeyboardInterrupt: 

## Export model

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

/home/tuchsanai/anaconda3/envs/llm/lib/python3.12/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


## Test model

In [ ]:
from transformers import  StoppingCriteria, StoppingCriteriaList

class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_ids:list):
        self._i = 0
        self.keywords = keywords_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        text = tokenizer.decode(
                input_ids[0],
                skip_special_tokens=True
        )
        if text.strip().endswith("</answer>"):
            return True
        if self._i % 50 == 0:
            print(text)
            print("-" * 16)
        self._i += 1
        if input_ids[0][-1] in self.keywords:

            return True
        return False

stop_words = ['</answer>']


stop_ids = [tokenizer.encode(w) for w in stop_words]
stop_criteria = KeywordsStoppingCriteria(stop_ids)

stopping_criteria = StoppingCriteriaList([stop_criteria])


text = """<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Act as a doctor and response the input question from a patient in Thai language with XML format <answer>Your answer</answer>
### Input:
คือผมอยากทราบว่า อาการที่ผมเป็นตอนนี้คือกรดไหลย้อน หรือ เป็นสัญญาณของพิษสุนัขบ้าครับ ผมมีอาการ เเน่นๆ อึดอัดที่คอ เเล้วก็ กลืนน้ำลายลำบากครับ ก่อนหน้านี้มีไข้ต่ำ ปวดหัวนิดหน่อยครับ ช่วง 1 เดือนก่อน ผมทำงานเดินทางโดยจักรยานครับ ทางผ่านมีสุนัขอยู่ตามทางเยอะมากๆ (เเต่จากที่เห็นไม่ได้เห่าเเละไล่ตามผมครับ) เเล้วมาพึ่งมาเป็นอาการดังกล่าวช่วงนี้ครับ ผมจึงไม่เเน่ใจว่าเป็นกรดไหลย้อนหรือเป็นสัญญานอาการเเรกเริ่มของพิษสุนัขบ้าหรอครับ ผมอ่านเเล้วเห็นอาการคล้ายๆกันครับคุณ รบกวนด้วยนะครับ
### Response:"""

device = "cuda:0"

lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_to_save.generate(
    **inputs, max_new_tokens=400,
    stopping_criteria=stopping_criteria,
    pad_token_id=tokenizer.eos_token_id,
    no_repeat_ngram_size=10,
    forced_eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_p=0.95
)



display(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/tuchsanai/anaconda3/envs/llm/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Act as a doctor and response the input question from a patient in Thai language with XML format <answer>Your answer</answer>
### Input:
คือผมอยากทราบว่า อาการที่ผมเป็นตอนนี้คือกรดไหลย้อน หรือ เป็นสัญญาณของพิษสุนัขบ้าครับ ผมมีอาการ เเน่นๆ อึดอัดที่คอ เเล้วก็ กลืนน้ำลายลำบากครับ ก่อนหน้านี้มีไข้ต่ำ ปวดหัวนิดหน่อยครับ ช่วง 1 เดือนก่อน ผมทำงานเดินทางโดยจักรยานครับ ทางผ่านมีสุนัขอยู่ตามทางเยอะมากๆ (เเต่จากที่เห็นไม่ได้เห่าเเละไล่ตามผมครับ) เเล้วมาพึ่งมาเป็นอาการดังกล่าวช่วงนี้ครับ ผมจึงไม่เเน่ใจว่าเป็นกรดไหลย้อนหรือเป็นสัญญานอาการเเรกเริ่มของพิษสุนัขบ้าหรอครับ ผมอ่านเเล้วเห็นอาการคล้ายๆกันครับคุณ รบกวนด้วยนะครับ
### Response:
----------------


'<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nAct as a doctor and response the input question from a patient in Thai language with XML format <answer>Your answer</answer>\n### Input:\nคือผมอยากทราบว่า อาการที่ผมเป็นตอนนี้คือกรดไหลย้อน หรือ เป็นสัญญาณของพิษสุนัขบ้าครับ ผมมีอาการ เเน่นๆ อึดอัดที่คอ เเล้วก็ กลืนน้ำลายลำบากครับ ก่อนหน้านี้มีไข้ต่ำ ปวดหัวนิดหน่อยครับ ช่วง 1 เดือนก่อน ผมทำงานเดินทางโดยจักรยานครับ ทางผ่านมีสุนัขอยู่ตามทางเยอะมากๆ (เเต่จากที่เห็นไม่ได้เห่าเเละไล่ตามผมครับ) เเล้วมาพึ่งมาเป็นอาการดังกล่าวช่วงนี้ครับ ผมจึงไม่เเน่ใจว่าเป็นกรดไหลย้อนหรือเป็นสัญญานอาการเเรกเริ่มของพิษสุนัขบ้าหรอครับ ผมอ่านเเล้วเห็นอาการคล้ายๆกันครับคุณ รบกวนด้วยนะครับ\n### Response:'

In [ ]:
display(tokenizer.decode(outputs[0], skip_special_tokens=True))

'<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nAct as a doctor and response the input question from a patient in Thai language with XML format <answer>Your answer</answer>\n### Input:\nคือผมอยากทราบว่า อาการที่ผมเป็นตอนนี้คือกรดไหลย้อน หรือ เป็นสัญญาณของพิษสุนัขบ้าครับ ผมมีอาการ เเน่นๆ อึดอัดที่คอ เเล้วก็ กลืนน้ำลายลำบากครับ ก่อนหน้านี้มีไข้ต่ำ ปวดหัวนิดหน่อยครับ ช่วง 1 เดือนก่อน ผมทำงานเดินทางโดยจักรยานครับ ทางผ่านมีสุนัขอยู่ตามทางเยอะมากๆ (เเต่จากที่เห็นไม่ได้เห่าเเละไล่ตามผมครับ) เเล้วมาพึ่งมาเป็นอาการดังกล่าวช่วงนี้ครับ ผมจึงไม่เเน่ใจว่าเป็นกรดไหลย้อนหรือเป็นสัญญานอาการเเรกเริ่มของพิษสุนัขบ้าหรอครับ ผมอ่านเเล้วเห็นอาการคล้ายๆกันครับคุณ รบกวนด้วยนะครับ\n### Response:'

# (Optional) Push trained model to your Huggingface account

In [ ]:
model_to_save.push_to_hub("typhoon-med")
tokenizer.push_to_hub("typhoon-med")
model_to_save.config.push_to_hub("typhoon-med")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-665e2939-38941208543277a978679b3d;4de0941e-a605-4c56-a0ac-2128e327dea5)

Invalid username or password.